In [24]:
import numpy as np
from matplotlib import pyplot as plt
from time import time
from math import factorial as fact
from numpy import sqrt
import commpy
from random import shuffle
from scipy.stats import norm
π = np.pi
tol = 1e-14
num_trials = 20
num_points = 500
max_iters = 100000

In [25]:
plt.rcParams["figure.dpi"] = 300

# Packet Error Trials
### The Process

Using the Binary Phase Shift Keying (BPSK) and Cyclic Redundancy Check (CRC), this attempts to graph/calculate the effect on average packet percentage error of a few variables: 1) Packet Length (in bits), 2) Sigma in the Rayleigh with AWGN channel.  
Messages are sent as strings of 0s and 1s.

In [26]:
# Define Universal Variables for system
num_packets = 100
packet_len = 51                                       # Includes the single checking bit
N = 7                                                 # Number of antenna
λ = 1                                                 # Wavelength
σ2_awgn = 1                                           # Noise/AWGN variance
σ2_ray = 1                                            # Noise/Fading variance

# Generates the signal
def gen_sig(packet_len,num_packets) :
    '''
    Generates an arbitrary signal to transmit num_packets of length packet_len (symbols are 1 and -1).
    
    Inputs :
        packet_len (int)  - Number of bits in a packet (includes the checking bit)
        num_packets (int) - Number of packets in message
        
    Output :
        to_transmit (num_packets,packet_len) array - The arbitrary signal
    '''
    to_transmit = np.random.randint(0,high=2,size=num_packets*packet_len).reshape(numm_packets,packet_len)
    to_transmit[:,-1] = np.sum(to_transmit[:,:-1],axis=1) % 2
    return 2*to_transmit-1

def packet_errs(pack_len,num_pack,rec_sig,true_sig) :
    '''
    Checks a received signal to determine the observed and true packet errors.
    Signal is inputed using symbols of 1 and -1, but checked using symbols of 1 and 0.
    
    Inputs :
        pack_len (int)                     - Number of bits in a packet (includes the checking bit)
        num_pack (int)                     - Number of packets in message
        rec_sig (num_pack,pack_len) array  - The received signal (corrected)
        true_sig (num_pack,pack_len) array - The sent signal
        
    Outputs :
        perc_err_per (float) - The perceived packet error percentage
        true_err_per (float) - The true packet error percentage
    '''
    true_sig = true_sig == 1
    rec_sig = rec_sig == 1
    perc_err_per = np.sum(np.sum(rec_sig[:,:],axis=1)%2,axis=0) / num_pack
    err_mask = rec_sig == true_sig
    pack_err_mask = np.sum(err_mask[:,:],axis=1) != 0
    true_err_per = np.sum(pack_err_mask,axis=0) / num_pack
    return perc_err_per, true_err_per

def dist(k,θ,r,N,λ) :
    '''
    Calculates the distance from the array element (middle of the antenna array is origin) to the transmitter
    which is r away from the origin.
    
    Inputs:
        k (int)       - index of antenna
        θ (rad)       - angle off from orthogonal to array
        r (unit-less) - distance to transmitter from origin
        N (int)       - number of antenna + 1
        λ (?)         - wavelength (c = fλ where c is the speed of light and f is the frequency)
        
    Output:
        Distance from antenna to transmitter
    '''
    dx = k*λ/2 - (N-1)*λ/4
    x = np.sin(θ)*r
    y = np.cos(θ)*r
    return sqrt((x-dx)**2+y**2)

def dst(θ,r,N,λ) :
    ds = np.zeros(N).astype(complex)
    for k in range(N) :
        ds[k] = np.exp(dist(k,θ,r,N,λ)*2*π*1j)
    ds = ds/np.linalg.norm(ds)
    return ds

def weights(θ,N,λ) :
    '''
    Calculates the coefficients of the beam equation
    
    Inputs:
        θ (rad) - angle off from orthogonal to array (off x-axis)
        N (int) - number of antenna +1
    
    Output:
        a_k for the beam form equation Σ_1^N <a_k(θ), x_k> where x_k is the volatge response of antenna k
        NOTE: This vector is normalized (why? I don't know)
    '''
    ϕ = π*np.sin(θ)
    bs = np.zeros(N).astype(complex)
    for k in range(N) :
        bs[k] = np.exp(ϕ*k*1j)
    return bs/np.linalg.norm(bs)

def gen_beam(θ,r):
    '''
    Generates a beam of the form [power_levels (n,) array, weights (N,) array] for a beam aiming in the
    direction of θ
    
    Inputs :
        θ (rad) - The angle off the x-axis for the beam
        
    Output :
        beam (as described above)
    '''
    if θ < 0 :
        θ += 2*π
    angles = np.linspace(0,2*π,num_points)
    ak = weights(θ,N,λ)
    sig_beam = [abs(dst(angle,r,N,λ)@ak) for angle in angles]
    beam = [sig_beam,ak]
    return beam

def plp(signal_beam,noise_beams,var_awgn=1,var_ray=1,h_sig=None,pack_len=10,dist_fad_exp=None,verbose=True) :
    '''
    Calculates the estimated SINR, BER, and PLP (packet loss percentage)
        for certain locations based on transmission beams.
    Assumes a Rayleigh channel with BPSK modulation and CRC.
    
    Input :
        Beams are of the form [power_levels (n,) array, weights (L,) array]
        signal_beam  - beam  - The signal beam
        noise_beams  - list  - List of noise beams (each beam like signal beam)
        var_awgn     - float - White noise variance
        var_ray      - float - Fading variance of the Rayleigh channel
        h_sig        - array - Fading from the Rayleigh channel [CN(0,var_ray) distributed random variable]
        pack_len     - int   - Number of bits per packet (including the checking bit)
        dist_fad_exp - float - Exponent for the (optional) distance attenuation fading
        verbose      - bool  - Whether or not you want an update every 100 iterations
        
    Output :
        SINR -  (n,n) array  - Signal to Interference and Noise Ratio
        BER  -  (n,n) array  - Bit Error Rate (for BPSK under https://www.unilim.fr/pages_perso/vahid/notes/ber_awgn.pdf)
        PLP  -  (n,n) array  - Packet Loss Percentage
    '''
    sig_beam = signal_beam[0]
    sig_weights = signal_beam[1]
    n_beams = [beam[0] for beam in noise_beams]
    n_weights = [beam[1] for beam in noise_beams]
    K = len(n_weights)
    n = len(sig_beam)
    L = sig_weights.shape[0]
    r = np.linspace(tol,max(sig_beam),n)
    SINR = np.zeros((n,n))
    if h_sig is None :
        h_sig = np.random.normal(loc=np.array([0,0]),scale=np.array([var_ray,var_ray]),size=(L,2))
        h_sig = 1/sqrt(2)*(h_sig[:,0] + 1j*h_sig[:,1])
    sig_const = abs(h_sig @ sig_weights)**2
    noise_const = [abs(h_sig[i] @ weight[i])**2 for i in range(K)]
    for j in range(n) : # Iterating over θ
        sig_pow = sig_beam[j]
        noise_pows = [beam[j] for beam in n_beams]
        if dist_fad_exp :
            SINR[j,:] = [((sig_pow/(r[i]**dist_fad_exp))*sig_const)/(var_awgn + sum([(noise_pows[k]/(r[i]**dist_fad_exp))*noise_const[k] for k in range(K)])) for i in range(n)]
        else :
            SINR[j,:] = [(sig_pow*sig_const)/(var_awgn + sum([noise_pows[k]*noise_const[k] for k in range(K)])) for i in range(n)]
        if verbose and (j+1) % 100 == 0 :
            print(f'{j+1}th iteration complete.')
    mask = SINR >= max(sig_beam)
    SINR[mask] = max(sig_beam)
    #BER = 1 - norm.cdf(np.sqrt(2*SINR*abs(h_sig@h_sig)))
    PLP = 1 - (norm.cdf(np.sqrt(2*SINR*abs(h_sig@h_sig))))**pack_len
    #SINR = 10*np.log(SINR)/np.log(10)
    return PLP

def get_angle_range(θ,r) :
    '''
    Determines [θ_min,θ_max] for sending the packets.
    
    Input :
        θ (rad) - The angle off the x-axis to Bob's location.
    
    Output :
        θ_min, θ_max (rad) - The minimum and maximum angle to choose for beamforming.
    '''
    optimal_beam = gen_beam(θ,r)[0]
    # Get values that are greater than half power
    mask = optimal_beam >= 0.5*max(optimal_beam)
    # Do 2π to 4π to avoid problems, fix it later.
    θs = np.linspace(2*π,4*π,num_points)*mask
#     print(f'To BOB: {θ}\n',θs)
    # Get the interval
    int_found = False
    iters = 0
    while not int_found and iters <= num_points :
        θ_min_ind = next((i for i, x in enumerate(θs) if x != 0), None)
#         print(f'First: {θ_min_ind}')
        #This is to handle the beam beginning with the interval (loops back)
        if θ_min_ind is not None and θ_min_ind == 0 and θs[-1] != 0 :
            ϕs = θs[::-1]
            θ_min_ind = -next((i for i, x in enumerate(ϕs) if x == 0), 0)
#             print(f'Update first: {θ_min_ind}')
        elif θ_min_ind is None :
            return θ, None # SOMETHING WENT WRONG
        if θ_min_ind >= 0 :
            θ_max_ind = next((i for i, x in enumerate(θs[θ_min_ind:]) if x == 0), -θ_min_ind) + θ_min_ind - 1
        else : 
            θ_max_ind = next((i for i, x in enumerate(θs) if x == 0), None)
            if θ_max_ind is None :
                raise ValueError('Interval could not be found.') # SOMETHING WENT WRONG
            else :
                θ_max_ind -= 1
#         print(θ_max_ind)
#         print(θs[θ_min_ind],θs[θ_max_ind])
#         print(θ+2*π)
        if θ_min_ind < 0 :
            θ_min = θs[θ_min_ind] - 4*π
        else :
            θ_min = θs[θ_min_ind] - 2*π
        θ_max = θs[θ_max_ind] - 2*π
        if (θ >= θ_min and θ <= θ_max) or ((θ-2*π) >= θ_min and (θ-2*π) <= θ_max) :
            int_found = True
        else :
            if θ_min_ind < 0 :
                θs[θ_min_ind:] = 0
                θs[:θ_max_ind+1] = 0
            elif θ_max_ind == -1 :
                θs[θ_min_ind:] = 0
            else :
                θs[θ_min_ind:θ_max_ind+1] = 0
#             print('Wrong Interval')
        iters += 1
    if θ_max == 0 :
        θ_max = 2*π
#     print(θ_min,θ_max)
    return θ_min,θ_max

def get_angles(θ_min,θ_max,num_packs=num_packets) :
    '''
    Determines the angles for each packet
    
    Input :
        θ_min (rad)     - The smallest valid angle off the x-axis to Bob's location.
        θ_max (rad)     - The largest valid angle off the x-axis to Bob's location.
        num_packs (int) - The number of packets to send (a.k.a. angles to pick)
        
    Output :
        angles (list) - The beam angles for sending the packets (ordered)
    '''
    # Make sure inputs are reasonable.
    assert (num_packs > 0),"Number of packets must be a positive."
    assert (int(num_packs) == num_packs), "Number of packets must be an integer."
    # Get length of 1/4 interval.
    α = (θ_max - θ_min)/4
    # Pick random 1/4 of the angles w/in first 1/4
    first_quartile = np.random.uniform(θ_min,θ_min+α,size=(num_packs//4))
    # Pick random 1/4 of the angles w/in last 1/4
    fourth_quartile = np.random.uniform(θ_max-α,θ_max,size=(num_packs//4))
    # Put the remaining 1/2 in the middle (since beam is almost symetrical, this doesn't need to be broken up).
    middle = np.random.uniform(θ_min+α,θ_max-α,size=(num_packs-2*(num_packs//4)))
    # Set up angles array (it's possible the first and last 1/4 are empty).
    if len(first_quartile) > 0 :
        angles = np.hstack((first_quartile,middle,fourth_quartile))
    else :
        angles = middle
    # Any negative angles, switch to be positive.
    neg_angles = angles < 0
    angles[neg_angles] += 2*π
    # Change to list so they can be randomized.
    angles = list(angles)
    shuffle(angles)
    return angles

#def send_packs()

In [27]:
def run_trial(num_trials=1,max_requests=5,verbose=False):
    '''
    Holy crap, I need to write a docstring.
    '''
    assert num_times > 0, "Cannot run a non-positive number of trials."
    assert int(num_times) == num_times, "Cannot run a non-integer number of trials."
    # Available angles between 0 and 2π
    θ = np.linspace(0,2*π,num_points)
    # Distance as a percentage power/max distance
    r = np.linspace(0,1,num_points)
    R,Θ = np.meshgrid(r,θ)
    # Randomly selected position for Bob, at least 50% out.
    Bobs_pos = (np.random.randint(0,num_points),np.random.randint(num_points//2,num_points))
    # Get the exact angle toward Bob.
    θ_to_bob = θ[Bobs_pos[0]]
    r_to_bob = r[Bobs_pos[1]]
    # Get min and max angles
    θ_min, θ_max = get_angle_range(θ_to_bob)
    # A little out of place, but report where Bob is located.
    if verbose :
        print(f'Bob is located at r={r[Bobs_pos[1]]}, θ={θ[Bobs_pos[0]]}.')
    # Number of trials that didn't result in Bob receiving every packet.
    non_converge = 0
    # Record how long each trial took.
    times = np.zeros(num_trials)
    # Record how many iterations/sets of feedback requests were needed.
    iterations = np.zeros(num_trials)
    # Record how many positions received all packets.
    num_rec_all = np.zeros(num_trials)
    # Begin trials loop.
    for trial in range(num_trials) :
        # Array to record for each position, for every packet, whether it was received or not.
        rec_at = np.zeros((num_points,num_points,num_packets))
        # List of ones.  A one in the i^th position means the i^th packet still needs to be sent.
        packets_to_send = [1]*num_packets
        # Counter to track iterations (sets of feedback requests).
        iters = 0
        # Time the sending of the signal.
        start = time()
        # Send the packets.
        while 1 in packets_to_send and iters <= max_iters :
            # Get an angle for every packet that needs to be sent this iteration.
            angles = get_angles(θ_min,θ_max,num_packs=sum(packets_to_send))
            # Counter for how many packets have been sent already this iteration.
            j = 0
            # Iterate through every possible packet.
            for i in range(num_packets) :
                # See if the packet needs to be sent.
                if packets_to_send[i] == 1 :
                    # Get the angle for this packet.
                    ϕ = angles[j]
                    # Record that the packet was/will be sent.
                    j += 1
                    # Generate the beam for this angle.
                    beam = gen_beam(ϕ,r_to_bob)
                    # Generate the PLP for each position for this beam.
                    PLP = plp(beam,noise_beams=[],var_awgn=σ2_awgn,var_ray=σ2_ray,pack_len=packet_len,verbose=False)
                    # Determine whether packet was received or not.
                    received = np.random.uniform(0,high=1,size=(num_points,num_points))
                    # Remember, PLP is Packet Loss Percentage, so less than or equal to percentage results in Packet LOSS.
                    # Higher results in the packet being received successfully.
                    rec_at[:,:,i] += received > PLP
                    # We don't care about a packet received more than once, just whether or not it was received.
                    mask = rec_at[:,:,i] >= 1
                    rec_at[:,:,i][mask] = 1
                    # If Bob received it, change update the list so that we don't need to send it again.
                    if rec_at[Bobs_pos[0],Bobs_pos[1],i] == 1 :
                        packets_to_send[i] = 0
            # Iteration complete
            iters += 1
            # Print progress (every 10).
            if iters % 10 == 0 and verbose :
                print(f'{iters}th iteration complete.')
        # Stop the clock when all iterations are complete (i.e. Bob received the full signal).
        end = time()
        # Print out how long it took.
        if verbose :
            print(f'{iters} iterations and {end-start} seconds to send all packets.')
        # If there was a problem receiving the packets, print it out, and update stats.
        #     Time and Iterations are set to 0, while the number of nonconvergent angles is increased.
        if iters > max_iters :
            if verbose :
                print('Could not receive all the packets.')
            iters = 0
            non_converge += 1
            end = start
        # Record results.
        iterations[trial] = iters
        times[trial] = end-start
        num_rec_at_point = np.sum(rec_at,axis=2)
        # Count how many positions received every packet.
        mask = num_rec_at_point == num_packets
        num_rec_all[trial] = np.count_nonzero(mask)
        print(f'Trial {trial+1} complete.')
    return Bobs_pos, num_rec_all, iterations, times, non_converge, mask

In [28]:
#print(f'{np.count_nonzero(mask)} out of {num_points**2} ({np.count_nonzero(mask)/(num_points**2)}%) locations received all packets.')
# plt.polar(θ[Bobs_pos[0]],r[Bobs_pos[1]],'*',alpha=0.5,markersize=5)
# plt.pcolor(Θ,R,num_rec_at)
# plt.show()

In [29]:
bob_pos, nra, it, tim, no_con, last_mask = run_trial(num_trials)

Trial 1 complete.
Trial 2 complete.
Trial 3 complete.
Trial 4 complete.
Trial 5 complete.
Trial 6 complete.
Trial 7 complete.
Trial 8 complete.
Trial 9 complete.
Trial 10 complete.
Trial 11 complete.
Trial 12 complete.
Trial 13 complete.
Trial 14 complete.
Trial 15 complete.
Trial 16 complete.
Trial 17 complete.
Trial 18 complete.
Trial 19 complete.
Trial 20 complete.


In [30]:
for i in range(num_trials) :
    print(f'Trial {i+1} took {tim[i]} seconds to do {it[i]} iterations.')

Trial 1 took 74.40224885940552 seconds to do 15.0 iterations.
Trial 2 took 73.75080394744873 seconds to do 10.0 iterations.
Trial 3 took 76.24795293807983 seconds to do 14.0 iterations.
Trial 4 took 329.1850070953369 seconds to do 14.0 iterations.
Trial 5 took 1336.143099784851 seconds to do 13.0 iterations.
Trial 6 took 69.02371096611023 seconds to do 13.0 iterations.
Trial 7 took 77.68587684631348 seconds to do 13.0 iterations.
Trial 8 took 70.68722486495972 seconds to do 13.0 iterations.
Trial 9 took 72.61792802810669 seconds to do 12.0 iterations.
Trial 10 took 76.14591598510742 seconds to do 14.0 iterations.
Trial 11 took 78.86419415473938 seconds to do 12.0 iterations.
Trial 12 took 68.21598196029663 seconds to do 15.0 iterations.
Trial 13 took 83.99222588539124 seconds to do 16.0 iterations.
Trial 14 took 74.54430103302002 seconds to do 16.0 iterations.
Trial 15 took 74.33682084083557 seconds to do 14.0 iterations.
Trial 16 took 86.22233414649963 seconds to do 14.0 iterations.
T

In [31]:
print(nra)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [32]:
print(last_mask[bob_pos[0],bob_pos[1]])

True


In [121]:
%prun run_trial(num_trials)

Bob is located at r=0.845691382765531, θ=0.3210846199059705.
10th iteration complete.
20th iteration complete.
22 iterations and 1093.5925188064575 seconds to send all packets.
 